## 최종버전

In [3]:
# 0. module import
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc
import datetime
import time
import pandas as pd
import numpy as np
import pickle as pkl
from tqdm import tqdm

In [1]:


# 🌟 중간에 코드가 끊겨서 다시 재시작하나요? 그러면 restart = True 로 변경해주세요. 변경하지 않으면 처음부터 다시 수집되니 주의해주세요!

def func(start_idx, end_idx, restart = False, ind = 0):
    start_time = time.time()
    # 1. 데이터프레임 및 카테고리 정의
    df = pd.DataFrame(columns=['대분류', '소분류', '강의명', '난이도', '가격(현재가격)','가격(원래가격)', '총소요시간', '강의소개', '언어', '출처']) 

    categories = ["web-development", "data-science", "mobile-apps", "programming-languages", "game-development",
                  "databases", "software-testing", "software-engineering", "development-tools", "no-code-development"]

    # 2. current_info를 저장할 pkl 파일 정보 생성
    current_info = {"category": categories[start_idx], "page": 1, "last_page": "(확인예정)", "lec_num": 0}
    lec_count = current_info['lec_num']

    # 3. 에러 때문에 중간부터 다시 시작하는 상황 대비
    if restart == True:
        df = pd.read_csv(f"./udemy_{current_info['category']}_230719.csv") 
        with open('current_info.pickle', 'rb') as file:
            current_info = pkl.load(file)

        start_idx = categories.index(current_info['category'])
        if current_info['lec_num']==15: #강의가 16개(idx 0~15)이므로 마지막 강의면 다음 페이지로 넘어갑니다.
            current_info['page']+= 1
            current_info['lec_num'] = 0
        lec_count = current_info['lec_num'] + 1

    # 4. 불러온 변수 확인 후 크롤링 시작!
    print(f"크롤링 시작 : 👍 {current_info['category']} : 총 {current_info['last_page']}페이지 중 {current_info['page']}번째 페이지의 {lec_count}번째 강의부터 수집 시작 ")

    # 5. 카테고리(대분류) 돌기
    try:
        for category in categories[start_idx:(end_idx + 1)]:
            options = uc.ChromeOptions()
            driver = uc.Chrome(use_subprocess=True, options=options)
            last_page = current_info['last_page']
            if last_page == "(확인예정)":
                while True:
                    try:     # 한국어, 영어강의만 추출
                        driver.get(f"https://www.udemy.com/ko/courses/development/{category}/?lang=en&lang=ko&p=1&ratings=4.5&sort=popularity")
                        break
                    except Exception as e:
                        exception_name = type(e).__name__
                        print(f"🥲 네트워크가 연결이 잘 안돼요... Exception: {exception_name}")
                        time.sleep(3)
                driver.maximize_window()
                time.sleep(5)
                last_page = int(driver.find_element(By.XPATH, '//*[@aria-label="생략 부호"]/following-sibling::span').text) #마지막 페이지 번호
                current_info['last_page'] = last_page
            current_info['category'] = category
            with open('current_info.pickle', 'wb') as file: # pickle 파일로 저장
                pkl.dump(current_info, file)

            # 6. 1페이지부터 last_page까지 페이지별 강의(lec) 크롤링
            for page in tqdm(range(current_info['page'], last_page + 1)):
                current_info["page"] = page
                while True:
                    try:     
                        driver.get(f"https://www.udemy.com/ko/courses/development/{category}/?lang=en&lang=ko&p={page}&ratings=4.5&sort=popularity")
                        break
                    except Exception as e:
                        exception_name = type(e).__name__
                        print(f"🥲 네트워크가 연결이 잘 안돼요... Exception: {exception_name}")
                        time.sleep(3)
                with open('current_info.pickle', 'wb') as file:  # pickle 파일로 저장
                    pkl.dump(current_info, file)
                time.sleep(7) #3 -> 7초 수정
                lec_container = driver.find_elements(By.XPATH, '//*[@id="udemy"]/div[1]/div[2]/div/div/div')
                div_num = len(lec_container)
                lec_boxes = driver.find_elements(By.XPATH, f'//*[@id="udemy"]/div[1]/div[2]/div/div/div[{div_num}]/div[2]/div/div[2]/div/div[2]/div[2]/div')

                # 태그가 없는 경우, nan값을 할당하고 넘어가는 함수를 사용합니다.
                def find_element_nan(driver, path):
                    try:
                        element = driver.find_element(By.XPATH, path)
                        return element.text
                    except:
                        return np.nan

                # 7. 페이지별 강의 상세내용 크롤링 시작    
                url_list = [] #페이지마다 리셋
                for box in lec_boxes:
                    a_tag = box.find_element(By.TAG_NAME, 'a')
                    urls = a_tag.get_attribute('href')
                    try:
                        level_list = ["모든 수준", "초급자", "중급자", "전문가"]
                        details_level=box.text.split("\n")
                        levels = np.nan

                        for x in level_list:
                            if x in details_level:
                                levels = x
                                break

                    except IndexError:
                        levels = np.nan

                    try:
                        details=box.text.split("\n현재 가격\n")
                        now_prices = details[1].split('\n')[0]
                    except IndexError:
                        now_prices = np.nan

                    try:
                        raw_prices = details[1].split('\n')[2]
                    except IndexError:
                        raw_prices = np.nan

                    if urls != "https://udemy.wjtb.co.kr/insight/index?ref=right-rail&locale=ko_KR": #유데미 광고 페이지인 경우에는 크롤하지 않음
                        url_list.append((urls, levels, now_prices, raw_prices))

                for url_idx, (urls, levels, now_prices, raw_prices) in enumerate(url_list):
                    if url_idx >= lec_count:
                        while True:
                            try:     
                                driver.get(urls)
                                time.sleep(7) # 3 -> 7초 수정
                                break
                            except Exception as e:
                                exception_name = type(e).__name__
                                print(f"🥲 네트워크가 연결이 잘 안돼요... Exception: {exception_name}")
                                time.sleep(3)
                        current_info["lec_num"] = url_idx
                        with open('current_info.pickle', 'wb') as file: # pickle 파일로 저장
                            pkl.dump(current_info, file)

                        try:
                            hashtags_parent = driver.find_element(By.CLASS_NAME, 'topic-menu.topic-menu-condensed.ud-breadcrumb')
                            hashtags_a = hashtags_parent.find_elements(By.TAG_NAME, 'a')
                            hashtag = hashtags_a[-1].text
                        except:
                            hashtag = np.nan

                        title = find_element_nan(driver, '//h1[@data-purpose="lead-title"]')
                        level = levels
                        now_price = now_prices
                        raw_price = raw_prices
                        lec_time = find_element_nan(driver, '//span[@data-purpose="video-content-length"]')
                        body = find_element_nan(driver, '//div[@class="component-margin what-you-will-learn--what-will-you-learn--1nBIT"]')
                        language = find_element_nan(driver, '//div[@data-purpose="lead-course-locale"]')
                        url = urls

                        # 8. csv 형태로 추출
                        new_row = {'대분류': category, '소분류': hashtag, '강의명': title, '난이도': level, 
                                '가격(현재가격)': now_price, '가격(원래가격)': raw_price, '총소요시간': lec_time, 
                                '강의소개': body, '언어': language, '출처': url}
                        df = pd.concat([df, pd.DataFrame(new_row, index=[0])], ignore_index=True)
                        df.to_csv(f'./udemy_{category}_230719.csv', index=False)
                        ind += 1
                        if ind % 500 == 0:
                            driver.quit()
                            return ind, df, False
                lec_count = 0
        
            driver.quit()
            return 'end', df, False
    except KeyboardInterrupt:
        print(f"👍 {current_info['category']} : 총 {current_info['last_page']}페이지 중 {current_info['page']}번째 페이지의 {current_info['lec_num']}번째 강의까지 수집 완료 ")
        return ind, df, True

# 9. 총 실행 시간 출력
# print(len(df), '개 데이터 크롤링 완료')
# end_time = time.time()
# execution_time = end_time - start_time

# hours = int(execution_time // 3600)
# minutes = int((execution_time % 3600) // 60)
# seconds = int(execution_time % 60)
# print("작업 실행 시간: {}시간 {}분 {}초".format(hours, minutes, seconds))

In [4]:

start = 3
end = 3
i = 0
while i != 'end':
    start_time = time.time()
    if i == 0:
        i, df, b = func(start, end, restart = False, ind = i)
    else:
        i, df, b = func(start, end, restart = True, ind = i)
    
    if b == True:
        print('크롤링 중단')
        print(len(df), '개 데이터 크롤링 완료')
        print('i=',i,'부터 크롤링 진행하세요')
        
    print(len(df), '개 데이터 크롤링 완료')
    end_time = time.time()
    execution_time = end_time - start_time

    hours = int(execution_time // 3600)
    minutes = int((execution_time % 3600) // 60)
    seconds = int(execution_time % 60)
    print("작업 실행 시간: {}시간 {}분 {}초".format(hours, minutes, seconds))
    
    time.sleep(5)


크롤링 시작 : 👍 programming-languages : 총 (확인예정)페이지 중 1번째 페이지의 0번째 강의부터 수집 시작 


 42%|████████████████████████████████▎                                            | 31/74 [1:18:12<1:48:28, 151.37s/it]


500 개 데이터 크롤링 완료
작업 실행 시간: 1시간 18분 19초
크롤링 시작 : 👍 programming-languages : 총 74페이지 중 32번째 페이지의 4번째 강의부터 수집 시작 


 72%|████████████████████████████████████████████████████████▉                      | 31/43 [1:18:44<30:29, 152.42s/it]


1000 개 데이터 크롤링 완료
작업 실행 시간: 1시간 18분 45초
크롤링 시작 : 👍 programming-languages : 총 74페이지 중 63번째 페이지의 8번째 강의부터 수집 시작 


100%|█████████████████████████████████████████████████████████████████████████████████| 12/12 [26:35<00:00, 132.95s/it]


1172 개 데이터 크롤링 완료
작업 실행 시간: 0시간 26분 36초


In [8]:
b

'd'

In [4]:
start = 0
end = 0
i = 3055
while i != 'end':
    start_time = time.time()
    
    i, df = func(start, end, restart = True, ind = i)
        
    print(len(df), '개 데이터 크롤링 완료')
    end_time = time.time()
    execution_time = end_time - start_time

    hours = int(execution_time // 3600)
    minutes = int((execution_time % 3600) // 60)
    seconds = int(execution_time % 60)
    print("작업 실행 시간: {}시간 {}분 {}초".format(hours, minutes, seconds))
    time.sleep(5)

크롤링 시작 : 👍 web-development : 총 242페이지 중 193번째 페이지의 1번째 강의부터 수집 시작 


 54%|█████████████████████████████████████████▌                                   | 27/50 [1:11:10<1:00:37, 158.16s/it]


3500 개 데이터 크롤링 완료
작업 실행 시간: 1시간 11분 11초
크롤링 시작 : 👍 web-development : 총 242페이지 중 220번째 페이지의 14번째 강의부터 수집 시작 


100%|█████████████████████████████████████████████████████████████████████████████████| 23/23 [52:40<00:00, 137.41s/it]


ValueError: too many values to unpack (expected 2)

In [6]:
db = pd.read_csv('./test_udemy_databases_230717.csv')
db.head()

,대분류,소분류,강의명,난이도,가격(현재가격),가격(원래가격),총소요시간,강의소개,언어,출처
0,databases,MySQL,【한글자막】 실무 활용 중심의 MySQL 부트캠프,초급자,"₩17,000","₩88,000",16.5시간 주문형 동영상,배울 내용\n데이터 베이스 생성 및 활용하는 방법\n여러개의 테이블을 조회하는 복잡...,한국어,https://www.udemy.com/course/best-mysql-k/
1,databases,Database Programming,너무 쉬운 데이터베이스(MSSQL) 입문,초급자,"₩17,000","₩69,000",5시간 주문형 동영상,"배울 내용\n관계형 데이터베이스, 비관계형 데이터베이스의 개념\n쇼핑몰 데이터베이스...",한국어,https://www.udemy.com/course/so_easy_database/
2,databases,MySQL,처음 시작하는 MySQL DataBase,모든 수준,"₩23,000",NaN,6시간 주문형 동영상,배울 내용\n본 과정에서는 MySQL 데이터 베이스에 대해 개발자 측면 부분을 학습...,한국어,https://www.udemy.com/course/softcampus-mysql/
3,databases,SQL,데이터 분석 필수 과정! MS SQL 로 실무 적용하기,모든 수준,"₩59,000",NaN,3시간 주문형 동영상,배울 내용\n데이터를 읽고 수정하고 적재하고 계산하는 작업을 효율적으로 할 수 있음...,한국어,https://www.udemy.com/course/cloom-ms-sql/
4,databases,SQL,실무 SQL 완전정복 - SECTION 1 : SQL 시작하기,초급자,"₩23,000",NaN,2시간 주문형 동영상,배울 내용\nSQL이 왜 중요한지에 대해 이해합니다.\nSQL-Server를 설치하...,한국어,https://www.udemy.com/course/sql-49-section-1/


In [7]:
db.to_csv('./test_udemy_databases.csv', encoding = 'utf-8-sig')

In [8]:
db = pd.read_csv('./test_udemy_game-development_230717.csv')
db.head()

db.to_csv('./test_udemy_game-development.csv', encoding = 'utf-8-sig')

In [9]:
db = pd.read_csv('./udemy_software-testing_230718.csv')
db.head()

db.to_csv('./test_udemy_software-testing.csv', encoding = 'utf-8-sig')

In [11]:
db = pd.read_csv('./udemy_web-development_230719.csv')
db.head()

db.to_csv('./test_udemy_web-development.csv', encoding = 'utf-8-sig')

In [13]:
len(db)

3839